## Задание 1
n - Количество наблюдений\
k - количество переменных\
$X = (h w b)$

$Y = X * b + E$

$A = X^T X$

$\hat{b} = A^{-1} X^T Y$

$\epsilon = X \hat{b} - Y$

$\hat{\sigma^2} = \frac{\epsilon^T \epsilon}{n - k}$

Мы знаем, что оценки коэффициентов имеют распределению Сттьюдента с параметром (n - k), а оценка остаточной дисперсии распределена по $\chi^2$. Найдем доверительные интервалы.\

Для оценок коэффициентов:\
$b_i \in [\hat{b_i} - t_{\alpha/2} S^2; \hat{b_i} + t_{\alpha/2} S^2]$\
Где $t_{\alpha/2}$ - квантиль распределения Стьюдента с параметром (n - k), \
$S^2$ - стандартная ошибка

Для оценки дисперсии:\
$\frac{(n - k) \hat{\sigma^2}}{\sigma^2} \sim \chi^2(n - k)$

Коэффициент детерминации:\
$R^2 = \frac{(\sum_{i=1}^n (y_i - \overline{y}) (\hat{y_i}- \hat{\overline{y}}))^2}{\sum_{i}(y_i - \overline{y})^2 \sum_{i} (\hat{y_i} - \hat{\overline{y}})^2}$


In [106]:
import numpy as np
import pandas as pd
from scipy.stats import t
from scipy.stats import chi2
import math
from scipy.stats import norm
from scipy.stats import f

alpha = 0.05

In [107]:
df = pd.read_csv("resources/mobile_phones.csv")
h = df['sc_h']
w = df['sc_w']
b = df['battery_power']
y = df['mobile_wt']
n = len(h)
k = 4

names = ["Ошибка: ", "Высота: ", "Ширина: ", "Ёмкость: "]

In [108]:
print(len(h), len(w), len(b))
X = np.array([np.ones(n), h, w, b]) # Ошибка, высота, ширина, батарея
X = X.T
X_T = X.T
A = X_T @ X
A_1 = np.linalg.inv(A)
b_ = A_1 @ X_T @ y
Y_ = X @ b_
print("Оценки коэффициентов: ")
print("Высота: ", format(b_[1], 'f'))
print("Ширина: ", format(b_[2], 'f'))
print("Ёмкость: ", format(b_[3], 'f'))
print("Ошибка: ", format(b_[0], 'f'))

e = Y_ - y
sigma_ = (e.T @ e) / (n - k)
print("\nОценка для остаточной дисперсии: " ,format(sigma_, 'f') )

y_mean = np.mean(y)
Y_mean = np.mean(Y_)

R_2_ = ((y - y_mean) @ (Y_ - Y_mean).T)**2 / (sum((y - y_mean)**2) * sum((Y_ - Y_mean)**2))
print("\nКоэффициент детерминации: " , format(R_2_, 'f'))

SE = np.sqrt(np.diag(sigma_ * A_1))
t_crit = t.ppf(1 - alpha/2, df=n-k)
CIs = []
print("\nДоверительные интервалы для оценок коэффициентов: ")
for beta_j, se_j, name in zip(b_, SE, names):
    CI_low = beta_j - t_crit * se_j
    CI_high = beta_j + t_crit * se_j
    CIs.append((CI_low, CI_high))
    print(name, '[', CI_low, ';', CI_high, ']')

print("\nДоверительный интервал для оценки остаточной дисперсии: ")
CI_low  = ((n - k) * sigma_) / chi2.ppf(1 - alpha/2, n - k)
CI_high = ((n - k) * sigma_) / chi2.ppf(alpha/2, n - k)
print('[', CI_low, ';', CI_high, ']')


2000 2000 2000
Оценки коэффициентов: 
Высота:  -0.263548
Ширина:  -0.039549
Ёмкость:  0.000064
Ошибка:  143.640572

Оценка для остаточной дисперсии:  1253.557631

Коэффициент детерминации:  0.001164

Доверительные интервалы для оценок коэффициентов: 
Ошибка:  [ 137.05003112925934 ; 150.23111339170433 ]
Высота:  [ -0.6910460538769494 ; 0.16394971525216723 ]
Ширина:  [ -0.4529080511643385 ; 0.37381081927541626 ]
Ёмкость:  [ -0.003471465855209107 ; 0.003600426631026992 ]

Доверительный интервал для оценки остаточной дисперсии: 
[ 1179.2805226501582 ; 1335.1160518152021 ]


In [109]:
def check(data):
    rank_data = data.rank(method='average')
    rank_y = y.rank(method='average')
    m = len(rank_y)
    s = 0
    for i in range(m):
        s += (rank_y[i] - (n+1)/2) * (rank_data[i] - (n+1)/2)

    s *= 12/(n * (n**2-1))

    s0 = 0
    for i in range(n):
        s0 += (rank_y[i] - rank_data[i])**2

    p_value = 1 - norm.cdf(s, loc = 0, scale = 1/math.sqrt(m - 1))
    if p_value < alpha:
        print(f"Reject")
    else:
        print(f"Accept")

Для проверки первой и второй гипотезы используем тест Спирмена. (привет лабе 4)

Для проверки третьей гипотезы воспользуемся F-критерием для нескольких ограничений.

F-критерий, записанный через коэффициенты детерминации:

$R^2_F$ - коэффициент детерминации для всех переменных\
$R^2_R$ - коэффициент детерминации для линейной можели только с высотой и свободным членом.\
$n - 4$ - степень свободы для всей модели\
$n - 2$ - степень для модели с ограничениями\
Тогда статистика

$F = \frac{R^2_F - R^2_R}{(n -2) - (n -4)} * \frac{n-4}{1 - R^2_F}$

Упростим выражение:
$F = \frac{R^2_F - R^2_R}{(n -2) - (n -4)} * \frac{n-4}{1 - R^2_F} = \frac{n-4}{2} * \frac{R^2_F - R^2_R}{1 - R^2_F}$

In [110]:
print("Гипотеза 1 (зависимость массы от высоты)")
check(h)
print("Гипотеза 2 (зависимость массы от ширины)")
check(w)

R_2_f = ((y - y_mean) @ (Y_ - Y_mean).T)**2 / (sum((y - y_mean)**2) * sum((Y_ - Y_mean)**2))
X_R = np.array([np.ones(n), h]) # Ошибка, высота, ширина, батарея
X_R = X_R.T
X_R_T = X_R.T
A_R = X_R_T @ X_R
A_R_1 = np.linalg.inv(A_R)
b_R_ = A_R_1 @ X_R_T @ y
Y_R_ = X_R @ b_R_
Y_R_mean = np.mean(Y_R_)
R_2_r = ((y - y_mean) @ (Y_R_ - Y_R_mean).T)**2 / (sum((y - y_mean)**2) * sum((Y_R_ - Y_R_mean)**2))
print(R_2_f, R_2_r)
F = (n - 4)/2 * (R_2_f - R_2_r) / (1 - R_2_f)
F_crit = f.ppf(1 - alpha, 2, n - 4)
F_crit_hight = f.ppf(1 - alpha/2,  2, n - 4)
F_crit_low = f.ppf(alpha/2,  2, n - 4)
print("Гипотеза 3 (равенство коэффициентов нулю)")
if F < F_crit_hight and F > F_crit_low:
    print(f"Accept", F, "[", F_crit_low, ";", F_crit_hight, "]")
else:
    print(f"Reject", F, "[", F_crit_low, ";", F_crit_hight, "]")

Гипотеза 1 (зависимость массы от высоты)
Accept
Гипотеза 2 (зависимость массы от ширины)
Accept
0.0011644496581530272 0.0011461416384521024
Гипотеза 3 (равенство коэффициентов нулю)
Reject 0.018292704595135434 [ 0.025318129124950843 ; 3.6957054126390574 ]


## Задача 2
Однофакторный дисперсионный анализ\
$H_0 = \mu_1= \mu_2$

$I = 2$

$F = \frac{n-I}{I-1}*\frac{S^2_B}{S^2_W}$

$\frac{S^2_B}{I-1}$ - межгрупповая дисперсия

$S^2_B = \sum_i J_i (\overline{y_{i*}} - \overline{y}) = J_1 (\overline{y_{1*}} - \overline{y})^2 + J_2 (\overline{y_{2*}} - \overline{y})^2$

$\frac{S^2_W}{n-I}$ - внутригрупповая дисперсия

$S^2_W = \sum_{i, j} (y_{ij} - \overline{y_{i*}})^2$

$\overline{y_{i*}}$ - среднее по группе по фактору.

Если значение F - больше критического значения (квантиля 1-$\alpha$), то мы опровергаем гипотезу о равенстве средниих, иначе принмаем.

In [111]:
df1 = pd.read_csv("resources/sex_bmi_smokers.csv")
factor = df1['smoker']
bmi = df1['bmi']

bmi_yes = bmi[factor =="yes"]
bmi_no = bmi[factor =="no"]

J1 = len(bmi_yes)
J2 = len(bmi_no)
N = J1 + J2
I = 2

mean_yes = np.mean(bmi_yes)
mean_no  = np.mean(bmi_no)
mean_total = np.mean(bmi)

S_2_B = J1 * (mean_yes - mean_total)**2 + J2 * (mean_no - mean_total)**2

S_2_W = np.sum((bmi_yes - mean_yes)**2) + np.sum((bmi_no - mean_no)**2)

MSB = S_2_B / (I - 1)
MSW = S_2_W / (n - I)


F = MSB / MSW

F_crit_hight = f.ppf(1 - alpha/2, dfn=I-1, dfd=N-I)
F_crit_low = f.ppf(alpha/2, dfn=I-1, dfd=N-I)
if F < F_crit_hight and F > F_crit_low:
    print(f"Accept", F, "[", F_crit_low, ";", F_crit_hight, "]")
else:
    print(f"Reject", F, "[", F_crit_low, ";", F_crit_hight, "]")


Accept 0.02810365284595238 [ 0.0009824370879597643 ; 5.035230976717185 ]
